<font color='yellow'>
<h1><b>FlyMe - Développez un chatbot pour réserver des vacances</b></h1>

<h2>Projet 10 - Parcours Ingénieur IA - Openclassrooms</h2>

</font>

[GITHUB](https://github.com/blanchonnicolas/IA_Project10_Openclassrooms_Chatbot)<br>
[DATASET](https://www.microsoft.com/en-us/research/project/frames-dataset/#!download)<br>

**FlyMe** est une agence qui propose des voyages clé en main pour les particuliers ou les professionnels.<br>
<img src="files/FlyMe_image.png">


Fly Me a lancé un projet ambitieux de développement d’un chatbot pour aider les utilisateurs à choisir une offre de voyage.<br>
Je suis l’un des ingénieurs IA au sein de l’équipe de cette entreprise, et je suis mobilisé pour le *projet chatbot - Construction d'un MVP pour les collaborateurs internes*.<br>
Ce premier MVP devra **pouvoir identifier dans la demande de l’utilisateur les cinq éléments suivants** :

- ✅ Ville de départ</br>
- ✅ Ville de destination
- [ ] Date aller souhaitée du vol
- [ ] Date retour souhaitée du vol
- ✅ Budget maximum pour le prix total des billets

> **Note:** Si un des éléments est manquant, le chatbot devra pouvoir poser les questions pertinentes (en anglais) à l’utilisateur pour comprendre complètement sa demande.

***

La solution de Chatbot, reposera sur la **stack téchnique suivante**:

1. Microsoft Bot [Microsoft Bot Framework SDK v4 for Python](https://github.com/microsoft/botbuilder-python)
2. [Service cognitif LUIS d’Azure](https://azure.microsoft.com/fr-fr/services/cognitive-services/language-understanding-intelligent-service)
3. Service Web App d’Azure
4. Bot Framework Emulator
5. [Azure application insight](https://docs.microsoft.com/fr-fr/azure/azure-monitor/app/app-insights-overview)


***
Au travers de ce notebook, nous réalisons l'étude préparatoir des données:<br>
- [Lecture des données](#Lecture-des-données)<br>
- [Analyse des données](#Analyse-des-données)<br>
- [Nettoyage des données](#Nettoyage-des-données)<br>
- [Sauvegarde des données](#Sauvegarde-des-données)<br>


## Lecture des données

In [1]:
#Import functions createdon my side
from Functions_custo_by_Nico import *
Test_Imported_Functions()


Functions have been properly imported !


In [2]:
src_path = os.path.abspath(os.path.join("../IA_Project10_Openclassrooms_Chatbot"))
if src_path not in sys.path:
    sys.path.append(src_path)

#Build Data path
data_path = os.path.join(src_path, "dataset")
print(data_path)
for f in os.listdir(src_path):
    print("|_",f)

c:\Users\blanc\OpenClassrooms\IA_Project10_Openclassrooms_Chatbot\dataset
|_ .git
|_ botbuilder
|_ dataset
|_ EDA.ipynb
|_ FlyMe_BOT_MVP
|_ Functions_custo_by_Nico.py
|_ luis_util.py
|_ README.md
|_ Sources
|_ __pycache__


In [160]:
#Read data
data = pd.read_json(os.path.join(data_path, 'frames', 'frames.json'))
print("Dimensionnalité : ", data.shape)
data.head()

Dimensionnalité :  (1369, 5)


,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


## Analyse des données

Comme nous avons pu le voir ci-dessous, les données Frames sont au format JSON [Plus dinfo ici](https://www.microsoft.com/en-us/research/project/frames-dataset/download/).
Chaque dialogue BOT/User est constitué de **5 champs principaux**: 
- user_id
- wizard_id
- id
- labels
- turns

Nous allons ci-dessous **explorer les champs labels et turns**, afin de comprendre leur structure respective.
> Note: Les champs labels et turns affichent l'imbrication de dict


In [4]:
data.describe(include='all')


,user_id,turns,wizard_id,id,labels
count,1369,1369,1369,1369,1369
unique,11,1369,12,1369,16
top,U22K1SX9N,[{'text': 'I'd like to book a trip to Atlantis...,U21T9NMKM,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
freq,345,1,301,1,929


In [5]:
#Column Label : Segregate Json format and spread info in new dataframe columns
satisfaction_data = pd.json_normalize(data["labels"])
data["userSurveyRating"] = satisfaction_data["userSurveyRating"]
data["wizardSurveyTaskSuccessful"] = satisfaction_data["wizardSurveyTaskSuccessful"]
print("Il y a ", data[data["wizardSurveyTaskSuccessful"] == False].shape[0],
      "dialogues qui ont rendu un effet négatif.")
#data.describe(include='all')


Il y a  82 dialogues qui ont rendu un effet négatif.


In [6]:
# Cautious!!! Some values are not integer !!!
data["userSurveyRating"].unique()


array([4.  , 3.  , 2.  , 5.  , 1.  ,  nan, 4.8 , 4.5 , 4.99, 3.5 ])

In [64]:
#Column turns : Segregate Json format and spread info in new dataframe columns
entry_id = 0
data_turns = pd.DataFrame(data['turns'][entry_id])
data_turns

,text,labels,author,timestamp,db
0,I'd like to book a trip to Atlantis from Capri...,"{'acts': [{'args': [{'val': 'book', 'key': 'in...",user,1.471272e+12,NaN
1,"Hi...I checked a few options for you, and unfo...",{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471272e+12,{'result': [[{'trip': {'returning': {'duration...
2,"Yes, how about going to Neverland from Caprica...","{'acts': [{'args': [{'val': 'Neverland', 'key'...",user,1.471273e+12,NaN
3,I checked the availability for this date and t...,{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471273e+12,"{'result': [[], [], [], [], [], []], 'search':..."
4,I have no flexibility for dates... but I can l...,"{'acts': [{'args': [{'val': False, 'key': 'fle...",user,1.471273e+12,NaN
5,I checked the availability for that date and t...,{'acts': [{'args': [{'val': [{'annotations': [...,wizard,1.471273e+12,"{'result': [[]], 'search': [{'ORIGIN_CITY': 'A..."
6,I suppose I'll speak with my husband to see if...,"{'acts': [{'args': [], 'name': 'thankyou'}], '...",user,1.471273e+12,NaN


In [65]:
# Display turns values for a chosen entry_id (= raw of Frames dataset), and looking more specifically to first discussion exchange  [0]
data['turns'][entry_id][0]

{'text': "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}],
    'name': 'inform'},
   {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
     {'val': 'Caprica', 'key': 'or_city'},
     {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
     {'val': '8', 'key': 'n_adults'},
     {'val': '1700', 'key': 'budget'}],
    'name': 'inform'}],
  'active_frame': 1,
  'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}],
     'budget': [{'val': '1700.0', 'negated': False}],
     'dst_city': [{

> Note: Les champs turns affichent l'imbrication de dict. Nous developperons une fonction ci-dessous afin de comprendre sa structure

Dans un premiers temps, nous analysons le nombre d'échanges User / BOT comptabilisé dans notre jeu de données.

Ensuite, nous chercherons à **extraire les données qui nous intérresse**, afin de les réutiliser lors de l'entrainement de notre BOT (Utilisation de LUIS)

In [18]:
#Study number of User/Wizard authoring text from dialogue
dic_author = {}
for turns in data['turns']:
    for message in turns:
        author = message["author"]
        if not author in dic_author.keys():
            dic_author[author] = 1
        else:
            dic_author[author] += 1
dic_author


{'user': 10407, 'wizard': 9579}

In [126]:
# Etract values recognized by BOT and usefull for our application # In labels column: 'acts': [{'name':
interesting_name = []
for entry_id in range(0,1369):
    for dialog_step in [data['turns'][entry_id][:1]]: # We look into 2 first dialogs only
        for dialog in dialog_step:    
            if dialog["author"] == "user": # We catch only user dialog
                acts = dialog["labels"]["acts"] # We look into labels info from turns, and more precisely under "act" column (Exclusion of 'acts_without_refs')
                for act in acts:
                    name = act["name"]
                    interesting_name.append(name)

interesting_name_unique, interesting_name_count = np.unique(interesting_name, return_counts=True)
# for i in range(len(interesting_name_unique)):
#     print(f"Name : {interesting_name_unique[i]} occurence = {interesting_name_count[i]}")
name_analysis = pd.DataFrame(columns=['Name under acts', 'Occurence'])
name_analysis['Name under acts'] = interesting_name_unique
name_analysis['Occurence'] = interesting_name_count
name_analysis.sort_values(by=['Occurence'], ascending=False)

,Name under acts,Occurence
1,inform,2252
0,greeting,405
2,request,15
3,thankyou,1


In [127]:
# Etract values recognized by BOT and usefull for our application # In labels column: 'acts': [{'name':
interesting_entities = []
for entry_id in range(0,1369):
    for dialog_step in [data['turns'][entry_id][:1]]: # We look into 2 first dialogs only
        for dialog in dialog_step:    
            if dialog["author"] == "user": # We catch only user dialog
                acts = dialog["labels"]["acts"] # We look into labels info from turns, and more precisely under "act" column (Exclusion of 'acts_without_refs')
                for act in acts:
                    if act["args"]:
                        for arg in act["args"]:
                            if "key" in arg: #info structure like: {'val': 'Atlantis', 'key': 'dst_city'},{'val': 'Caprica', 'key': 'or_city'}
                                key = arg["key"]
                                interesting_entities.append(key)

interesting_entities_unique, interesting_entities_count = np.unique(interesting_entities, return_counts=True)
# for i in range(len(interesting_entities_unique)):
#     print(f"Entity : {interesting_entities_unique[i]} occurence = {interesting_entities_count[i]}")
entity_analysis = pd.DataFrame(columns=['Entity under acts["args"]', 'Occurence'])
entity_analysis['Entity under acts["args"]'] = interesting_entities_unique
entity_analysis['Occurence'] = interesting_entities_count
entity_analysis.sort_values(by=['Occurence'], ascending=False)

,"Entity under acts[""args""]",Occurence
8,intent,1136
5,dst_city,897
14,or_city,811
19,str_date,381
11,n_adults,282
2,budget,238
6,end_date,215
12,n_children,114
9,max_duration,91
3,category,31


In [128]:
# Extract values recognized by BOT and usefull for our application # In labels column: 'acts': [{'args': [{'val': 'book'
entry_id = 0 #Select data Frames raw by choosing entry_id

interesting_entities_value = {} # We want to consolidate intersting variables for our BOT
for dialog_step in [data['turns'][entry_id][:1]]: # We look into 2 first dialogs only
    for dialog in dialog_step:
        if dialog["author"] == "user": # We catch only user dialog
            acts = dialog["labels"]["acts"] # We look into labels info from turns, and more precisely under "act" column (Exclusion of 'acts_without_refs')
            for act in acts:
                #name = act["name"]
                if act["args"]:
                    for arg in act["args"]:
                        if "key" in arg and "val" in arg: #info structure like: {'val': 'Atlantis', 'key': 'dst_city'},{'val': 'Caprica', 'key': 'or_city'}
                            key = arg["key"]
                            val = arg["val"]                                                  
                        else:
                            continue
                        if key not in interesting_entities_value:
                            interesting_entities_value[key] = [key]
                        else:
                            continue
                        interesting_entities_value[key].append(val)
                #interesting_entities[key].append(name)
interesting_entities_value


{'intent': ['intent', 'book'],
 'dst_city': ['dst_city', 'Atlantis'],
 'or_city': ['or_city', 'Caprica'],
 'str_date': ['str_date', 'Saturday, August 13, 2016'],
 'n_adults': ['n_adults', '8'],
 'budget': ['budget', '1700']}

Comme expliqué [ici](https://learn.microsoft.com/fr-fr/azure/cognitive-services/luis/concepts/application-design), notre schéma d'application (LUIS) devra contenir les intentions et les entités pertinentes pour la réservation de voyage.
***

> Note: Nous conservons uniquement les premiers echanges "turns" pour chaque ligne du jeu de données Frames.json

On remarque que notre jeu de données Frames.json les informations suivantes :
 - Intention: Reservation (**Book : spécifié dès le premier échange : *labels column: 'acts': [{'args': [{'val': 'book'***)
 - Destination: Atlantis
 - Origine: Caprica
 - Date: 13 Aout 2016
 - nombre d'adultes: 8
 - Budget: 1700
 - Ref: Nous avons exclus les arguments retrouvons ici les Tuples imbriqués, qui font références au message antérieurs de l'utilisateur (Nous n'utiliserons pas cette information dans le cadre de notre projet V1)

***

Voici les informations clés suivantes à déterminer par notre BOT : 
- 'dst_city'    (Occurence: 897)
- 'or_city'     (Occurence: 811)
- 'str_date'    (Occurence: 381)
- 'end_date'    (Occurence: 215)
- 'budget'      (Budget: 238)

>Nous notons ainsi que d'autres entités pourrait être pertinement ajoutées (nombre d'adultes, nombre d'enfants, durée maximum de voyage)

## Nettoyage des données

### Convertir les informations de notre dataset pour le rendre compatible avec le format JSON attendu par LUIS

Nous devons concevoir le schéma d’application LUIS. notamment :
- Intentions avec des exemples d’énoncés (Book_flight)
- [Entités](https://learn.microsoft.com/fr-fr/azure/cognitive-services/luis/luis-concept-entity-types) (éléments de données extraits de l’énoncé, ex: destination, la date, la catégorie de ticket et la classe de voyage, le nombre de voyageurs)
- Fonctionnalités

Les entités LUIS permettent au BOT de comprendre les événements / intentions. Cela permet de collecter des informations supplémentaires, afin que le BOT puisse poser des questions et répondre plus intelligemment. 
le fichier FlightBooking.json d'origine contient les intentions : « Book Flight », « Cancel » et « None », avec un ensemble d’entités telles que « From.Airport » et « To.Airport ».

Nous chercherons ci-dessous à :
1. Extraire les informations demandées pour notre MVP.
2. Formatter ces informations comme attendu par LUIS
> Note: Une attention particulière a été portée afin de nettoyer certains caractères préjudiciable à l'entrainement, tels que certaines monnaies (budget: $, £, €)

In [13]:
#Here an example of Utterance content
# "utterances": [
#     {
#       "text": "book flight from london to paris on feb 14th",
#       "intent": "Book flight",
#       "entities": [
#         {
#           "entity": "To",
#           "startPos": 27,
#           "endPos": 31
#         },

In [149]:
# Extract values recognized by BOT and usefull for our application # In labels column: 'acts': [{'args': [{'val': 'book'
entry_id = 0 #Select data Frames raw by choosing entry_id

interesting_entities_value = {} # We want to consolidate intersting variables for our BOT
for dialog_step in [data['turns'][entry_id][:1]]: # We look into 2 first dialogs only
    for dialog in dialog_step:
        if dialog["author"] == "user": # We catch only user dialog
            acts = dialog["labels"]["acts"] # We look into labels info from turns, and more precisely under "act" column (Exclusion of 'acts_without_refs')
            for act in acts:
                #name = act["name"]
                if act["args"]:
                    for arg in act["args"]:
                        if "key" in arg and "val" in arg: #info structure like: {'val': 'Atlantis', 'key': 'dst_city'},{'val': 'Caprica', 'key': 'or_city'}
                            key = arg["key"]
                            val = arg["val"]                                                  
                        else:
                            continue
                        if key not in interesting_entities_value:
                            interesting_entities_value[key] = [key]
                        else:
                            continue
                        interesting_entities_value[key].append(val)
                #interesting_entities[key].append(name)
interesting_entities_value

{'intent': ['intent', 'book'],
 'dst_city': ['dst_city', 'Atlantis'],
 'or_city': ['or_city', 'Caprica'],
 'str_date': ['str_date', 'Saturday, August 13, 2016'],
 'n_adults': ['n_adults', '8'],
 'budget': ['budget', '1700']}

In [150]:
def extract_entities(acts):
    dst_city = None
    or_city = None
    str_date = None
    end_date = None
    budget = None
    intent = None
    for act in acts:
        arg = act['args']
        if len(arg) == 0:
            break
        for entity in arg:
            if 'dst_city' in entity.values() and 'val' in entity.keys():
                dst_city =  entity['val']
            if 'or_city' in entity.values() and 'val' in entity.keys():
                or_city =  entity['val']
            if 'str_date' in entity.values() and 'val' in entity.keys():
                str_date =  entity['val']
            if 'end_date' in entity.values() and 'val' in entity.keys():
                end_date =  entity['val']
            if 'budget' in entity.values() and 'val' in entity.keys():
                budget =  entity['val']
            if 'intent' in entity.values() and 'val' in entity.keys():
                intent = entity['val']
            
    return or_city, dst_city, str_date, end_date, budget, intent #, n_adults, n_children, seat

In [264]:
def convert_to_luis_json(range_ids = range(len(data)), include_greeting=False):
    json_labels={'text':'text', 'intent':'intent', 'entities':'entities', 'startPos':'startPos', 'endPos':'endPos', 'entity':'entity'}
    json_top = []
    for entry_id in range_ids:
        # We will use only the first sentence of the dialog --> [0]
        text = data['turns'][entry_id][0]['text']
        acts = data['turns'][entry_id][0]['labels']['acts']
        or_city, dst_city, str_date, end_date, budget, intent = extract_entities(acts) #, n_adults, n_children, seat

        if intent == "book":  
            intent = "Book flight"
        elif any([or_city, dst_city]): #, str_date, end_date, budget, n_adults, n_children, seat
            intent = "Book flight"
        # elif include_greeting==True: #Check if greeting bool from function launch
        #     if acts[0]["name"]=="greeting": #Add the Greeting, in case some of them are present in the first dialogs
        #         intent = "greeting"
        #     else:
        #         continue
        entry_dict = {}
        entry_dict[json_labels['text']] = text
        entry_dict[json_labels['intent']] = intent
        entities = []
        #for k, v in zip(["From", "To"], [or_city, dst_city]) : #Working OK
        #for k, v in zip(["From", "To", "budget"], [or_city, dst_city, budget]) :  #Working OK
        for entity_key, entity_value in zip(["From", "To", "str_date", "end_date", "budget"], [or_city, dst_city, str_date, end_date, budget]) :
        #for k, v in zip(["From", "To", "str_date", "end_date", "budget", "n_adults", "n_children", "seat"], [or_city, dst_city, str_date, end_date, budget, n_adults, n_children, seat]) :
            if entity_value is None or entity_value == -1 or entity_value == str(-1): #Skip entity if value -1
                continue
            if entity_key == "budget": #Remove currency and associated space characters
                for currency in ["$", "usd", "dollar", "dollars", "€", "euro", "euros", "£", "pound", "pounds", " "]:
                    entity_value = entity_value.lower().replace(currency,"")
            try:   
                entity_value = re.escape(entity_value) #Avoid regex : Return string with all non-alphanumerics backslashed
                match=(re.search(entity_value.lower(), text.lower())) #Find where characters match (positions)
                entity = {json_labels["entity"]:entity_key, json_labels["startPos"]:match.start(), json_labels["endPos"]:match.end() -1} #Create JSON for LUIS compatibility
                entities.append(entity)
            except AttributeError:
                continue
        entry_dict[json_labels['entities']] = entities
        json_top.append(entry_dict)
    return json_top

In [265]:
#Let's try how the conversion to Json compatible with LUIS works
#xxx = convert_to_luis_json(range(0,900), include_greeting=False)
xxx = convert_to_luis_json(range(0,10), include_greeting=False)  #range(len(data)) for full dataset
print(json.dumps(xxx, indent=2))

[
  {
    "text": "I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.",
    "intent": "Book flight",
    "entities": [
      {
        "entity": "From",
        "startPos": 41,
        "endPos": 47
      },
      {
        "entity": "To",
        "startPos": 27,
        "endPos": 34
      },
      {
        "entity": "str_date",
        "startPos": 52,
        "endPos": 76
      },
      {
        "entity": "budget",
        "startPos": 117,
        "endPos": 120
      }
    ]
  },
  {
    "text": "Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.",
    "intent": "Book flight",
    "entities": [
      {
        "entity": "From",
        "startPos": 44,
        "endPos": 54
      },
      {
        "entity": "To",
        "startPos": 59,
        "endPos": 68
      },
      {
        "entity": "budget",
        "startPos": 75,
        "endPos": 78
      }
    ]
  },
  {
    "text": "Hel

## Sauvegarde des données

### Split du fichier JSON en Train / Test, et sauvegarde JSON

Maintenant que nous savons récuperer les informations du fichier Frames.json, afin de le rendre compatible avec les services LUIS.
Nous allons procéder  l'extraction des informations, et leur sauvegarde:
- Intentions avec des exemples d’énoncés (Book_flight)
- [Entités](https://learn.microsoft.com/fr-fr/azure/cognitive-services/luis/luis-concept-entity-types) (éléments de données extraits de l’énoncé, ex: destination, la date, la catégorie de ticket et la classe de voyage, le nombre de voyageurs)

Nous modifierons le fichier FlightBooking.json en agissant sur la partie **utterances**, afin d'utiliser les information extraites de notre **jeu de données Frames.json**.

In [266]:
train_size = round(data.shape[0] * 0.9)
print(f"Train dataset size is {train_size}")
valid_size = data.shape[0] - train_size
print(f"Validation dataset size is {valid_size}")
train_json = convert_to_luis_json(range(0,train_size))
valid_json = convert_to_luis_json(range(train_size, train_size+valid_size))
with open('dataset/train.json', 'w') as file:
    json.dump(train_json, file)
    
with open('dataset/valid.json', 'w') as file:
    json.dump(valid_json, file)
    #json.dump(valid_json, file, indent=2)

Train dataset size is 1232
Validation dataset size is 137


#### Ajout des données d'entrainement au fichier FlightBooking.json


In [170]:
#OK : starting from native FlightBooking file or template0
#with open('dataset/FlightBooking.json', 'r') as file:
with open('dataset/FlightBooking_template0.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified0.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

In [27]:
#OK : starting from template1 FlightBooking created manually (From To Budget)
with open('dataset/FlightBooking_template1.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified1.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

In [31]:
#OK : starting from template1 FlightBooking created manually (From To Budget str_date end_date)
with open('dataset/FlightBooking_template2.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified2.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

In [267]:
#OK : starting from template1 FlightBooking created manually (From To Budget str_date end_date)
with open('dataset/FlightBooking_template3.json', 'r') as file:
    luis_config = json.load(file)

luis_config['utterances'] = train_json
#print(json.dumps(luis_config, indent=2))

with open('dataset/FlightBooking_modified3.json', 'w') as file:
    json.dump(luis_config, file, indent=1)

# Conclusions

Au travers de ce notebook, nous avons étudié notre jeu de données **Frames.json**, afin d'en extraire les informations nécessaires à l'entrainement de notre BOT, qui fonctionnera gràce au service d'Azure cognitifs services LUIS.

Le fichier d'entrainement ci-dessus sera utilisé afin d'entrainer nos services Azure, et exploiter au mieux les futures réponses de nos clients.